In [2]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder
import random
import timeAfter the user presses the photo button, the system will automatically upload photos to the server for strabismus diagnosis. For the convenience of users, there is no precise restriction on the distance to be photographed. In addition, the patient can upload multiple photos for strabismus detection, and when more than one of the photos is judged to be strabismic, the user will be returned a "strabismus" result, and vice versa, a "normal" result
import torchvision
import torch.nn as nn
# from tqdm import tqdm_notebook as tqdm
from PIL import Image, ImageFile
from torch.utils.data import Dataset
import torch
import torch.optim as optim
from torchvision import transforms
from torch.optim import lr_scheduler
import os
from torchvision import models

ImageFile.LOAD_TRUNCATED_IMAGES = True

# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ['CUDA_VISIBLE_DEVICE'] = "0,1,2,3"
device_ids = [0]  

# torch.cuda.set_device(0)
modellr = 2e-4#学习率
BATCH_SIZE = 256
EPOCHS = 200
DEVICE = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
class_name = ['    斜视','   正常眼',]

train_transforms = transforms.Compose(
    [
        transforms.Resize((224,48)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation((-2, 2)),
        # transforms.ColorJitter(contrast=[0.8,1.2]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        #transforms.Normalize(mean=[0.0,0.0,0.0], std=[4.0,4.0,4.0]),
    ]
)
test_transforms = transforms.Compose(
    [
        transforms.Resize((224, 48)), 
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        # transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5]),
        #transforms.Normalize(mean=[0.0,0.0,0.0], std=[4.0,4.0,4.0]),
    ]
)

def Open(x):
    I = Image.open(x).convert('RGB')
#     I = I.convert('L')
    return I

train_set = DatasetFolder("/disk/user/lyf/vit/train", loader=Open, extensions=("jpg","png"), transform=train_transforms)
val_set = DatasetFolder("/disk/user/lyf/vit/val", loader=Open, extensions=("jpg","png"), transform=test_transforms)
test_set = DatasetFolder("/disk/user/lyf/vit/test", loader=Open, extensions=("jpg","png"), transform=test_transforms)





In [88]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICE'] = "0,1,2,3"
device_ids = [0,1,2,3]  

In [89]:
# from timm.models import inception_resnet_v2
# model = inception_resnet_v2(pretrained=True ,num_classes = 2)

from dencenet import densenet121
model = densenet121(num_classes = 2)
pretrained_model_dict = torch.load('/disk/user/lyf/vit/densenet121.pth')

model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [47]:
tmp=torch.randn([2,3,256,64])   

model1=models.resnet18(pretrained=True)
class NET(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1=nn.Sequential(*list(model1.children())[:-4])
#         self.layer1=nn.Sequential(
#             nn.Conv2d(in_channels=3,out_channels=32,kernel_size=7,stride=2,padding=1),
#             nn.BatchNorm2d(32),
#             nn.ReLU(),
#             nn.MaxPool2d((3,3))
#         )
        self.layer2=nn.Sequential(
            nn.Conv2d(in_channels=128,out_channels=128,kernel_size=3,stride=1,padding=1),
            nn.Dropout2d(),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d((2,2)),
            nn.Conv2d(in_channels=64,out_channels=32,kernel_size=3,stride=1,padding=1),
            nn.Dropout2d(),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(in_channels=32,out_channels=16,kernel_size=3,stride=1,padding=1),
            nn.Dropout2d(),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d((2,2))
        )
        self.layer3=nn.Sequential(
            nn.Linear(32*16*3,256),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(256,2),
            nn.Softmax()
        )
        self.layer4=nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=16,kernel_size=8,stride=8),
            nn.Dropout2d(),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,stride=1,padding=1),
            nn.Dropout2d(),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d((2,2)),
            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1,padding=1),
            nn.Dropout2d(),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d((2,2)),
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,stride=1,padding=1),
            nn.Dropout2d(),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d((2,2))
        )

    def forward(self,x):
        y=self.layer4(x)
        x=self.layer1(x)
        x=self.layer2(x)
        x=torch.cat([x,y],dim=1)
        x=torch.flatten(x,1)
        x=self.layer3(x)
        # print(x.shape)  
        return x


model=NET()
# model = torch.nn.DataParallel(model, device_ids=device_ids)
# model = model.cuda(device = device_ids[0])
# if isinstance(model, torch.nn.DataParallel):
#    model = model.module
tmp2=model.layer4(tmp)
print(tmp2.shape)
tmp=model.layer1(tmp)
print(tmp.shape)
tmp=model.layer2(tmp)
print(tmp.shape)
tmp3=torch.cat([tmp,tmp2],dim=1).flatten()
print(tmp3.shape)
# print(model(tmp))

torch.Size([2, 128, 4, 1])
torch.Size([2, 128, 32, 8])
torch.Size([2, 64, 16, 4])


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 16 but got size 4 for tensor number 1 in the list.

In [ ]:
model1

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [91]:
list(model1.children())[:-5]

[Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False),
 Sequential(
   (0): BasicBlock(
     (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   )
   (1): BasicBlock(
     (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), pad

In [92]:
model = model.cuda(device=device_ids[0])

data_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, drop_last=False)
valid_data_loader = torch.utils.data.DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, drop_last=True)
test_data_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, drop_last=False)
criterion = nn.CrossEntropyLoss() #交叉熵函数


base_params = []
first_params=[]
for n,p in model.named_parameters():
    if 'layer1' not in n:
        first_params+=[p]
    else:
        base_params+=[p]

group_params=[
            {'params': base_params},
            {'params': first_params, 'lr': modellr * 0.0},
            ]
group_params2=[
            {'params': base_params},
            {'params': first_params, 'lr': modellr * 0.1},
            ]
# optimizer = optim.Adam(model.parameters(), lr=modellr,weight_decay=0.0001) #Adam优化器

# optimizer = optim.Adam(group_params, lr=modellr,weight_decay=0.0001)
# optimizer2 = optim.Adam(group_params2, lr=modellr,weight_decay=0.0001)
optimizer = optim.Adam(model.parameters(), lr=modellr, weight_decay=0.001)
cosine_schedule = optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=30, eta_min=1e-9)
# optimizer = nn.DataParallel(optimizer, device_ids=device_ids)


In [93]:
from sklearn.metrics import roc_curve, auc
def print_roc_curves(type,epoch_id, y_test, y_score):
    fpr,tpr,thre = roc_curve(y_test, y_score)
    ##计算auc的值，就是roc曲线下的面积
    auc_ = auc(fpr,tpr)
    ##画图
    plt.plot(fpr,tpr,color = 'darkred',label = 'roc area:(%0.2f)'%auc_)
    plt.plot([0,1],[0,1],linestyle = '--')
    plt.xlim([0,1])
    plt.ylim([0,1])
    plt.xlabel('fpr')
    plt.ylabel('tpr')
    plt.title('roc_curve')
    plt.legend(loc = 'lower right')
    if(epoch_id % 10 == 0):
        f = plt.gcf()
        f.savefig('/disk/user/lyf/vit/resultch/RocCurve/{}{}.jpg'.format(type,epoch_id+1))
        f.clear()  #释放内存
    # plt.show()

In [94]:
from torch import nn
import torch
from torch.nn import functional as F

#focal_loss func, L = -α(1-yi)**γ *ce_loss(xi,yi)
class focal_loss(nn.Module):
    def __init__(self, alpha=0.4, gamma=2, num_classes = 2, size_average=True):
        super(focal_loss,self).__init__()
        self.size_average = size_average
        if isinstance(alpha,list):
            assert len(alpha)==num_classes   
            self.alpha = torch.Tensor(alpha)
        else:
            assert alpha<1  
            self.alpha = torch.zeros(num_classes)
            self.alpha[0] += alpha
            self.alpha[1:] += (1-alpha)

        self.gamma = gamma

    def forward(self, preds, labels):
        # assert preds.dim()==2 and labels.dim()==1
        preds = preds.view(-1,preds.size(-1))
        self.alpha = self.alpha.to(preds.device)
        preds_softmax = F.softmax(preds, dim=1) 
        preds_logsoft = torch.log(preds_softmax)
        
        #focal_loss func, Loss = -α(1-yi)**γ *ce_loss(xi,yi)
        preds_softmax = preds_softmax.gather(1,labels.view(-1,1)) 
        preds_logsoft = preds_logsoft.gather(1,labels.view(-1,1))
        self.alpha = self.alpha.gather(0,labels.view(-1))
        # torch.pow((1-preds_softmax), self.gamma) 为focal loss中 (1-pt)**γ
        loss = -torch.mul(torch.pow((1-preds_softmax), self.gamma), preds_logsoft) 

        loss = torch.mul(self.alpha, loss.t())
        if self.size_average:
            loss = loss.mean()
        else:
            loss = loss.sum()
        return loss
    
criterion=focal_loss()

In [96]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    modellrnew = modellr * (0.1 ** (epoch // 30))
    print("lr:", modellrnew)
    for param_group in optimizer.param_groups:
        param_group['lr'] = modellrnew

def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
     torch.backends.cudnn.deterministic = True  # 为了确定算法，保证得到一样的结果。
     torch.backends.cudnn.enabled = True  # 使用非确定性算法
     torch.backends.cudnn.benchmark = True  # 是否自动加速。
# 设置随机数种子
setup_seed(20)

best_acc = 0
train_loss, valid_loss, valid_acc, train_acc, test_loss, test_acc = [], [], [], [], [], []

# from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
def print_confusion_matrix(y_true, y_pred, type, epoch_id):
    # C = confusion_matrix(y_true, y_pred, labels=[0, 1])
    # plt.matshow(C, cmap=plt.cm.Reds)
    # plt.colorbar()
    # for i in range(len(C)):
    #     for j in range(len(C)):
    #         plt.annotate(C[j, i], xy=(i, j), horizontalalignment='center', verticalalignment='center')
    # plt.ylabel('True label')
    # plt.xlabel('Predicted label')
    # f = plt.gcf()
    # f.savefig('result2/matrix/{}{}.jpg'.format(type,epoch_id))
    # f.clear()  #释放内存
    # # plt.show()
    pass

@torch.no_grad()
def Test(epoch):
    print('Test')
    #tk1 = tqdm(test_data_loader, total=int(len(test_data_loader)))
    acc_num, sum_num, loss_sum = 0, 0, 0
    class_num = [0,0]
    class_sum = [0,0]
    total_labels = total_outputs = scores = []
    TP = FP = FN = TN = 0
    model.eval()
    for ti,(inputs, labels) in enumerate(valid_data_loader):
        # inputs = inputs.to(device, dtype=torch.float)
        # labels = labels.to(device, dtype=torch.long)
        inputs, labels = inputs.cuda(device=device_ids[0]), labels.cuda(device=device_ids[0])
        # outputs = model(inputs)
        outputs = model(inputs)
#         print('label :',labels.shape())
#         print('outputs :', outputs.shape())
        total_outputs = np.append(total_outputs, outputs.argmax(-1).cpu().numpy())
        total_labels = np.append(total_labels, labels.cpu().numpy())
        scores = np.append(scores, outputs.max(1)[0].cpu().numpy())
        loss = criterion(outputs, labels)
        for i in range(BATCH_SIZE): 
            if i == len(outputs.argmax(-1)):
                break
            if outputs.argmax(-1)[i].item() == labels[i].item():
                class_num[labels[i].item()] += 1
            class_sum[labels[i].item()] += 1
        acc_num += (labels==outputs.argmax(-1)).sum()
        sum_num += labels.size(0)
    acc = acc_num / sum_num
    class_acc = np.array(class_num) / np.array(class_sum)
    for i in range(2):
        print('Test class_{}: {}  ACC: {:.4f}  {}'.format(i, class_sum[i], class_acc[i], class_name[i][:-4]))
    print('Test Acc: {:.4f}'.format(acc))
    Precision = TP / (TP + FP)
    Recall = TP / (TP+FN)
    F1 = 0
    if((Precision + Recall) != 0):
        F1 =  2 * (Precision * Recall) /  (Precision + Recall)
    print('Precision: {}'.format(Precision) )
    print('Recall:    {}'.format(Recall))
    print('F1_score:  {}'.format(F1))
    fpr,tpr,thre = roc_curve(total_labels, scores)
    Auc = auc(fpr,tpr)
    print('Auc:  {}'.format(Auc))
    if  epoch % 10 == 0:       
        print('------------confusion_matrix------------- ')
    #     print(total_labels.shape)
    #     print(total_outputs.shape)
        print_confusion_matrix(total_labels , total_outputs, 'test', epoch)
        print_roc_curves('test_',epoch, total_labels, scores)
        
    global best_acc
    test_loss.append(loss_sum)
    test_acc.append(acc.cpu())

import matplotlib.pyplot as plt
def plotTheCurve(epoch_id):
    plt.figure(figsize=(8,6))
    plt.plot([i for i in range(len(train_loss))],train_loss,'',label="train_loss")
    plt.plot([i for i in range(len(test_loss))],test_loss,'',label="valid_loss")
    plt.title('loss')
    plt.legend(loc='upper right')
    plt.xlabel('epoch')
    plt.ylabel('')
    plt.grid(len(train_loss))
    # f = plt.gcf()
    # f.savefig('result2/LossCurve/test_{}.jpg'.format(epoch_id))
    # f.clear()  #释放内存
    plt.show()
    
    plt.figure(figsize=(8,6))
    plt.plot([i for i in range(len(train_acc))],train_acc,'',label="acc")
    plt.plot([i for i in range(len(test_acc))],test_acc,'',label="acc")
    plt.title('acc')
    plt.legend(loc='upper right')
    plt.xlabel('epoch')
    plt.ylabel('')
    plt.grid(len(train_loss))
    # f = plt.gcf()
    # f.savefig('result2/AccCurve/test{}.jpg'.format(epoch_id))
    # f.clear()  #释放内存
    plt.show()

import warnings
warnings.filterwarnings("ignore")

since = time.time()

for epoch in range(0, EPOCHS):
    print('Epoch {}/{}'.format(epoch, EPOCHS))
    print('-' * 30)
    adjust_learning_rate(optimizer, epoch)
#     print("lr:", scheduler.get_lr()[0])
    model.train()
    running_loss = 0.0
    #tk0 = tqdm(data_loader, total=int(len(data_loader)))
    counter = 0
    acc_num, sum_num = 0, 0
    class_num = [0,0]
    class_sum = [0,0]
    for bi,(inputs, labels) in enumerate(data_loader):
        inputs, labels = inputs.cuda(device=device_ids[0]), labels.cuda(device=device_ids[0])
        optimizer.zero_grad()
        # with torch.set_grad_enabled(True):
        # outputs = model(inputs)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
#             scheduler.step()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        for i in range(BATCH_SIZE): 
            if i == len(outputs.argmax(-1)):
                break
            if outputs.argmax(-1)[i].item() == labels[i].item():
                class_num[labels[i].item()] += 1
            class_sum[labels[i].item()] += 1
        acc_num += (labels==outputs.argmax(-1)).sum()
        sum_num += labels.size(0)
        counter += 1
        #tk0.set_postfix(loss=(running_loss / (counter * data_loader.batch_size)))
    class_acc = np.array(class_num) / np.array(class_sum)
    epoch_loss = running_loss / len(data_loader)
    acc = acc_num / sum_num
    for i in range(2):
        print('Train class_{}: {}  ACC: {:.4f}  {}'.format(i, class_sum[i], class_acc[i], class_name[i]))
    print('Training Loss: {:.4f}, Training Acc: {:.4f}'.format(epoch_loss, acc))
    cosine_schedule.step()
    # Valid()
    Test(epoch)
    train_loss.append(epoch_loss)
    train_acc.append(acc.cpu())
    if  epoch % 10 == 0:
        plotTheCurve(epoch)
        torch.save(model.state_dict(), "save_model/{}.pth".format(epoch))

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

print('-'*50)
print('-'*50)
print('-'*50)
# logger.info('-'*50)
# logger.info('-'*50)
# logger.info('-'*50)
# test_set = MyDataset("/disk/user/lyf/vit/test", transform=train_transforms)
test_data_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, drop_last=False)
Test(1)

Epoch 0/200
------------------------------
lr: 0.0002
Train class_0: 580  ACC: 0.4431     正常眼
Train class_1: 948  ACC: 0.7384      斜视
Training Loss: 24.5800, Training Acc: 0.6263
Test


ZeroDivisionError: division by zero